# Rewriting "if" statement with "where" statement

In [4]:
import numpy as np

In [4]:
def lala(z, x):
    if z<0: 
        out = -(z+x)
    else: 
        out = z+x
    return real(out)

def kaka(z, x):
    return np.where(z<0, -(z+x), (z+x))

In [5]:
zvec = np.arange(-2.5, 2, 1)
xvec = np.arange(-2.5, 2, 1)
zm, xm = np.meshgrid(zvec, xvec, sparse=False, indexing='ij')

In [9]:
# This shows that if statement doesn't work with arrays
lala(zm,xm)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
# np.where works
kaka(zm,xm)

array([[ 5.,  4.,  3.,  2.,  1.],
       [ 4.,  3.,  2.,  1., -0.],
       [ 3.,  2.,  1., -0., -1.],
       [-2., -1.,  0.,  1.,  2.],
       [-1.,  0.,  1.,  2.,  3.]])

# Rewriting m and alpha functions 
# ( so that they works with arrays, not just numbers )

## Function Dependency: 
## psi_s > kappa > alpha > m > Omega > (eta, zeta, nu)

In [2]:
# Skip this cell if CSR2D/ is already added under PYHTONPATH 
import os, sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)  

In [1]:
# Import basic functions which don't need re-write 
from csr2d.core import nu, eta, zeta, Omega

In [2]:
from numpy import real, sin
beta = 0.999

In [22]:
# Modification of m and alpha functions from core.py:
# The m function now returns absolute value to prevernt slightly negative number for square roots in alpha
# The alpha function now uses the "where" statement, and the arguments for square roots 

def m(z, x, beta):
    """
    Eq. (A2) from Ref[1]
    """
    out = -nu(x,beta)/3 + ( zeta(z,x,beta)/3 + nu(x,beta)**2/36 ) *Omega(z,x,beta)**(-1/3) + Omega(z,x,beta)**(1/3)
    return abs(out)

def alpha(z, x, beta):
    """
    Eq. (A4) from Ref[1]
    """
    try:
        #out1 = 1/2*(-(2*m(z,x,beta))**(1/2) + ( -2*(m(z,x,beta) + nu(x,beta)) + 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2) )**(1/2))
        #out2 = 1/2*( (2*m(z,x,beta))**(1/2) + ( -2*(m(z,x,beta) + nu(x,beta)) - 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2) )**(1/2))
        out1 = 1/2*(-(2*m(z,x,beta))**(1/2) + ( abs(-2*(m(z,x,beta) + nu(x,beta)) + 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2)) )**(1/2))
        out2 = 1/2*( (2*m(z,x,beta))**(1/2) + ( abs(-2*(m(z,x,beta) + nu(x,beta)) - 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2)) )**(1/2))
        return np.where(z<0, real(out1), real(out2))
    except ZeroDivisionError:
        return 0
    
def a1(z, x, beta):
    """
    Eq. (A4) from Ref[1]
    """
    try:
        arg1 = 2*m(z,x,beta)
        out1 = 1/2*(-np.sqrt(arg1) + np.sqrt( abs(-2*(m(z,x,beta) + nu(x,beta)) + 2*eta(z,x,beta)/np.sqrt(arg1)) ))
        out2 = 1/2*( np.sqrt(arg1) + np.sqrt( abs(-2*(m(z,x,beta) + nu(x,beta)) - 2*eta(z,x,beta)/np.sqrt(arg1)) ))
        return np.where(z<0, real(out1), real(out2))
    except ZeroDivisionError:
        return 0

In [10]:
zvec2

array([-1.e-05,  0.e+00,  1.e-05])

In [11]:
zvec2**(1/2)

/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


array([       nan, 0.        , 0.00316228])

In [9]:
np.sqrt(zvec2)

/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


array([       nan, 0.        , 0.00316228])

In [14]:
gamma = 500
rho = 1
sigmax = 10E-6
sigmaz = 10E-6

N = 10
dz = 1.0E-4/N
dx = 1.0E-4/N
zvec2 = np.arange(-1*sigmaz, 2*sigmaz, dz)
xvec2 = np.arange(-1*sigmax, 2*sigmax, dx)
zm, xm = np.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

In [15]:
# This works
m(zm,xm,beta)

array([[4.80159739e-05, 4.84748426e-05, 4.89393474e-05],
       [2.16840434e-19, 0.00000000e+00, 2.16840434e-19],
       [4.80159739e-05, 4.84748426e-05, 4.89393474e-05]])

In [20]:
# This gives error messages because of 1/sqrt(2m), and m ~ 0 at (z,x) ~ (0,0)
# I want the output at (0,0) to be zero, but it's nan now...
t1 = alpha(zm,xm,beta)

/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in power
/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in multiply
/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in power
/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply


In [23]:
t2 = a1(zm,xm,beta)

/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


In [27]:
np.nan_to_num(t2)

array([[-3.75391426e-06, -5.00250125e-06, -3.75390019e-06],
       [ 5.49503070e-02,  0.00000000e+00,  5.46761022e-02],
       [ 9.79583525e-03,  9.84130062e-03,  9.88961230e-03]])

In [25]:
# It however works with a single number because of the "try" and "except" statement
# Linearity doesn't carry over to array...

alpha(0,0,beta)

0

In [32]:
def kappa(z,x,beta):
    """
    Eq. (13) from Ref[1] with argumaent zeta = 0
    """
    return ( x**2 + 4*(1+x) *sin(alpha(z,x,beta))**2 )**(1/2)
def psi_s(z, x, beta):
    """
    2D longitudinal potential
    Eq. (23) from Ref[1] with no constant factor (e*beta**2/2/rho**2).
    Ref[1]: Y. Cai and Yuantao. Ding, PRAB 23, 014402 (2020)
    """
    try:
        out = (cos(2*alpha(z,x,beta)) - 1/(1+x)) / (kappa(z,x,beta) - beta*(1+x)*sin(2*alpha(z,x,beta)))
    except ZeroDivisionError:
        out = 0
        #print(f"Oops!  ZeroDivisionError at (z,x)= ({z:5.2f},{x:5.2f}). Returning 0.")
    return out

In [33]:
# The error message and "nan" carry over, which will cause further problems at the convolution step 
psi_s(zm,xm,beta)

<ipython-input-26-0fb7d6db598f>:19: RuntimeWarning: divide by zero encountered in power
  out1 = 1/2*(-(2*m(z,x,beta))**(1/2) + ( abs(-2*(m(z,x,beta) + nu(x,beta)) + 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2)) )**(1/2))
<ipython-input-26-0fb7d6db598f>:19: RuntimeWarning: invalid value encountered in multiply
  out1 = 1/2*(-(2*m(z,x,beta))**(1/2) + ( abs(-2*(m(z,x,beta) + nu(x,beta)) + 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2)) )**(1/2))
<ipython-input-26-0fb7d6db598f>:20: RuntimeWarning: divide by zero encountered in power
  out2 = 1/2*( (2*m(z,x,beta))**(1/2) + ( abs(-2*(m(z,x,beta) + nu(x,beta)) - 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2)) )**(1/2))
<ipython-input-26-0fb7d6db598f>:20: RuntimeWarning: invalid value encountered in multiply
  out2 = 1/2*( (2*m(z,x,beta))**(1/2) + ( abs(-2*(m(z,x,beta) + nu(x,beta)) - 2*eta(z,x,beta)*(2*m(z,x,beta))**(-1/2)) )**(1/2))


array([[-4.99883392e-01, -2.50250376e-06,  4.99866827e-01],
       [-2.18928793e+01,             nan, -2.19290813e+01],
       [-9.78691125e+00, -9.38702757e+00, -8.98845403e+00]])

## Trying with an mesh which skips (z,x)=(0,0)

In [34]:
zvec2 = np.arange(-1.5*sigmaz, 1.5*sigmaz, dz)
xvec2 = np.arange(-1.5*sigmax, 1.5*sigmax, dx)
zm, xm = np.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

In [35]:
# Making sure z=0 is not a grid point
zm

array([[-1.5e-05, -1.5e-05, -1.5e-05],
       [-5.0e-06, -5.0e-06, -5.0e-06],
       [ 5.0e-06,  5.0e-06,  5.0e-06]])

In [36]:
m(zm,xm,beta)

array([[1.03720755e-04, 1.04660309e-04, 1.05610342e-04],
       [1.22280701e-05, 1.23488854e-05, 1.24712824e-05],
       [1.22280701e-05, 1.23488854e-05, 1.24712824e-05]])

In [37]:
# So as long as z is not close to zero, the array function works...
alpha(zm,xm,beta)

array([[-5.63087667e-06, -7.29563863e-06, -7.29562090e-06],
       [ 3.11180543e-06, -1.87695537e-06, -1.87695185e-06],
       [ 4.94842679e-03,  4.96780823e-03,  4.99237624e-03]])

In [38]:
psi_s(zm,xm,beta)

array([[-0.49988753, -0.16666683,  0.16665725],
       [-1.49664849, -0.49987925,  0.49987097],
       [-6.33195519, -5.39045187, -4.44676955]])